<a href="https://colab.research.google.com/github/ozturkergin/ozturkergin/blob/main/TEFAS_Fon_Analiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -U marshmallow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.3 MB/s eta 0:00:00


In [58]:
from datetime import date
from marshmallow import Schema, fields, EXCLUDE, pre_load, post_load

class InfoSchema(Schema):
    date = fields.Date(data_key="TARIH", allow_none=True)
    price = fields.Float(data_key="FIYAT", allow_none=True)
    code = fields.String(data_key="FONKODU", allow_none=True)
    title = fields.String(data_key="FONUNVAN", allow_none=True)
    market_cap = fields.Float(data_key="PORTFOYBUYUKLUK", allow_none=True)
    number_of_shares = fields.Float(data_key="TEDPAYSAYISI", allow_none=True)
    number_of_investors = fields.Float(data_key="KISISAYISI", allow_none=True)

    # pylint: disable=no-self-use
    # pylint: disable=unused-argument
    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        # Convert milliseconds Unix timestamp to date
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hool(self, output_data, **kwargs):
        # Fill missing fields with default None
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    # pylint: enable=no-self-use
    # pylint: enable=unused-argument

    class Meta:
        unknown = EXCLUDE

class BreakdownSchema(Schema):
    date = fields.Date(data_key="TARIH", allow_none=True)
    tmm = fields.Float(data_key="TMM (%)", allow_none=True)
    repo = fields.Float(data_key="R", allow_none=True)
    code = fields.String(data_key="FONKODU", allow_none=True)
    other = fields.Float(data_key="D", allow_none=True)
    stock = fields.Float(data_key="HS", allow_none=True)
    eurobonds = fields.Float(data_key="EUT", allow_none=True)
    bank_bills = fields.Float(data_key="BB", allow_none=True)
    derivatives = fields.Float(data_key="T", allow_none=True)
    reverse_repo = fields.Float(data_key="TR", allow_none=True)
    term_deposit = fields.Float(data_key="VM", allow_none=True)
    treasury_bill = fields.Float(data_key="HB", allow_none=True)
    foreign_equity = fields.Float(data_key="YHS", allow_none=True)
    government_bond = fields.Float(data_key="DT", allow_none=True)
    precious_metals = fields.Float(data_key="KM", allow_none=True)
    commercial_paper = fields.Float(data_key="FB", allow_none=True)
    fx_payable_bills = fields.Float(data_key="DB", allow_none=True)
    foreign_securities = fields.Float(data_key="YMK", allow_none=True)
    private_sector_bond = fields.Float(data_key="OST", allow_none=True)
    participation_account = fields.Float(data_key="KH", allow_none=True)
    foreign_currency_bills = fields.Float(data_key="DÖT", allow_none=True)
    asset_backed_securities = fields.Float(data_key="VDM", allow_none=True)
    real_estate_certificate = fields.Float(data_key="GAS", allow_none=True)
    foreign_debt_instruments = fields.Float(data_key="YBA", allow_none=True)
    government_lease_certificates = fields.Float(data_key="KKS", allow_none=True)
    fund_participation_certificate = fields.Float(data_key="FKB", allow_none=True)
    government_bonds_and_bills_fx = fields.Float(data_key="KBA", allow_none=True)
    private_sector_lease_certificates = fields.Float(data_key="OSKS", allow_none=True)

    # pylint: disable=no-self-use
    # pylint: disable=unused-argument
    @pre_load
    def pre_load_hook(self, input_data, **kwargs):
        # Convert milliseconds Unix timestamp to date
        seconds_timestamp = int(input_data["TARIH"]) / 1000
        input_data["TARIH"] = date.fromtimestamp(seconds_timestamp).isoformat()
        return input_data

    @post_load
    def post_load_hook(self, output_data, **kwargs):
        # Replace None values with 0 for float fields
        output_data = {
            k: v
            if not (isinstance(self.fields[k], fields.Float) and v is None)
            else 0.0
            for k, v in output_data.items()
        }
        # Fill missing fields with default None
        output_data = {f: output_data.setdefault(f) for f in self.fields}
        return output_data

    # pylint: enable=no-self-use
    # pylint: enable=unused-argument

    class Meta:
        unknown = EXCLUDE

In [81]:
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union
import requests
import pandas as pd
import math

class tefas_get:

    root_url = "http://www.fundturkey.com.tr"
    detail_endpoint = "/api/DB/BindHistoryAllocation"
    info_endpoint = "/api/DB/BindHistoryInfo"
    headers = {
        "Connection": "keep-alive",
        "X-Requested-With": "XMLHttpRequest",
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 "
            "(KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
        ),
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Origin": "http://www.fundturkey.com.tr",
        "Referer": "http://www.fundturkey.com.tr/TarihselVeriler.aspx",
    }

    def __init__(self):
        self.session = requests.Session()
        _ = self.session.get(self.root_url)
        self.cookies = self.session.cookies.get_dict()

    def fetch(
        self,
        start: Union[str, datetime],
        end: Optional[Union[str, datetime]] = None,
        name: Optional[str] = None,
        columns: Optional[List[str]] = None,
    ) -> pd.DataFrame:

        start_date_initial = datetime.strptime(start, "%Y-%m-%d")
        end_date_initial = datetime.strptime(end or start, "%Y-%m-%d")
        counter = 1
        start_date = start_date_initial
        end_date = end_date_initial

        range_date = end_date_initial - start_date_initial
        range_interval = 90
        #print(range_date.days)

        info_schema = InfoSchema(many=True)
        # detail_schema = BreakdownSchema(many=True)
        merged = pd.DataFrame()

        if range_date.days > range_interval :
          counter = range_date.days / range_interval
          counter = math.ceil(counter)
          end_date = start_date + timedelta(days=range_interval)

        while counter > 0:
          counter -= 1
          #print(counter)
          #print(start_date)
          #print(end_date)

          data = {
              "fontip": "YAT",
              "bastarih": _parse_date(start_date),
              "bittarih": _parse_date(end_date),
              "fonkod": name.upper() if name else "",
          }

          # General info pane
          info = self._do_post(self.info_endpoint, data)
          info = info_schema.load(info)
          info = pd.DataFrame(info, columns=info_schema.fields.keys())
          #print(info)

          # Portfolio breakdown pane
          #detail = self._do_post(self.detail_endpoint, data)
          #detail = detail_schema.load(detail)
          #detail = pd.DataFrame(detail, columns=detail_schema.fields.keys())
          #print(detail)

          merged = pd.concat([merged, info])

          # Return only desired columns
          merged = merged[columns] if columns else merged

          if counter > 0 :
            start_date = end_date + timedelta(days=1)
            end_date = end_date + timedelta(days=range_interval)
            if end_date > end_date_initial :
              end_date = end_date_initial

        return merged

    def _do_post(self, endpoint: str, data: Dict[str, str]) -> Dict[str, str]:
        # TODO: error handling. this is quiet fishy now.
        response = self.session.post(
            url=f"{self.root_url}/{endpoint}",
            data=data,
            cookies=self.cookies,
            headers=self.headers,
        )
        return response.json().get("data", {})

def _parse_date(date: Union[str, datetime]) -> str:
    if isinstance(date, datetime):
        formatted = datetime.strftime(date, "%d.%m.%Y")
    elif isinstance(date, str):
        try:
            parsed = datetime.strptime(date, "%Y-%m-%d")
        except ValueError as exc:
            raise ValueError(
                "Date string format is incorrect. " "It should be `YYYY-MM-DD`"
            ) from exc
        else:
            formatted = datetime.strftime(parsed, "%d.%m.%Y")
    else:
        raise ValueError(
            "`date` should be a string like 'YYYY-MM-DD' "
            "or a `datetime.datetime` object."
        )
    return formatted

In [87]:
tefas = tefas_get()

assets = ['AKU','ST1','AFO', 'OJK', 'AFT', 'ALE', 'AOY', 'GOH']

date_start = '2022-08-12'
date_end = '2023-08-12'

df = pd.DataFrame()

for fon in assets:
#  print(fon)
  fetched_data = tefas.fetch(start=date_start, end=date_end, name=fon, columns=["date", "price"])
  fetched_data.set_index(['date'], inplace=True)
  new_df = pd.DataFrame(fetched_data)
  new_df.rename(columns = {'price': fon}, inplace=True)
  df = pd.concat([df, new_df])
  df.fillna(0,inplace=True)

df = df.groupby(['date']).sum(numeric_only=True)
df

,AKU,ST1,AFO,OJK,AFT,ALE,AOY,GOH
date,,,,,,,,
2022-08-12,0.174308,39.689022,0.226821,2.092273,0.167419,3.077159,0.198003,2.222873
2022-08-15,0.173493,39.501464,0.226846,2.093382,0.170249,3.081678,0.199524,2.228606
2022-08-16,0.172319,39.422981,0.227866,2.093155,0.170935,3.083182,0.199587,2.259864
2022-08-17,0.176514,40.117497,0.227500,2.084777,0.170095,3.084680,0.196873,2.254359
2022-08-18,0.180963,41.302809,0.227384,2.088417,0.168132,3.086200,0.195989,2.261445
...,...,...,...,...,...,...,...,...
2023-08-07,0.441054,121.636020,0.364926,3.339700,0.297736,3.822003,0.232858,6.773318
2023-08-08,0.444325,123.311277,0.365390,3.347146,0.296714,3.824891,0.229571,6.852885
2023-08-09,0.440821,121.415084,0.369228,3.380838,0.293649,3.827622,0.227439,6.932659


In [88]:
pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.6 MB/s eta 0:00:00


In [89]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

mu = expected_returns.mean_historical_return(df)
print(mu)

S = risk_models.sample_cov(df)

#optimize for max sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print("Max Sharpe Results")
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

ef = EfficientFrontier(mu, S)
weights = ef.max_quadratic_utility()
cleaned_weights = ef.clean_weights()
print("Max Quadratic Utility")
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

AKU    1.545066
ST1    1.984476
AFO    0.599005
OJK    0.588425
AFT    0.763994
ALE    0.244532
AOY    0.161223
GOH    2.043168
dtype: float64
Max Sharpe Results
OrderedDict([('AKU', 0.00219), ('ST1', 0.0), ('AFO', 0.0), ('OJK', 0.00517), ('AFT', 0.00033), ('ALE', 0.98771), ('AOY', 0.0), ('GOH', 0.00461)])
Expected annual return: 25.8%
Annual volatility: 1.0%
Sharpe Ratio: 23.92
Max Quadratic Utility
OrderedDict([('AKU', 0.0), ('ST1', 0.0), ('AFO', 0.0), ('OJK', 0.0), ('AFT', 0.0), ('ALE', 0.0), ('AOY', 0.0), ('GOH', 1.0)])
Expected annual return: 204.3%
Annual volatility: 30.4%
Sharpe Ratio: 6.66


(2.0431678887725186, 0.3038646136052305, 6.658122723697411)